In [1]:
import pandas as pd
orders=pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [2]:
users=pd.read_json("users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [3]:
import sqlite3
conn=sqlite3.connect(":memory:")
cursor=conn.cursor()
with open("restaurants.sql","r")as f:
    sql_script=f.read()
cursor.executescript(sql_script)
restaurants=pd.read_sql("Select * from restaurants",conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [5]:
merged=orders.merge(users,on="user_id",how="left")
final_df=merged.merge(restaurants,on="restaurant_id",how="left")
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [6]:
final_df.to_csv("final_food_delivery_dataset.csv",index=False)

In [16]:
import pandas as pd
final_df=pd.read_csv("final_food_delivery_dataset.csv")

In [8]:
final_df.groupby("order_date").size()

order_date
01-01-2023    29
01-01-2024    27
01-02-2023    25
01-03-2023    37
01-04-2023    32
              ..
31-05-2023    33
31-07-2023    27
31-08-2023    24
31-10-2023    30
31-12-2023    29
Length: 366, dtype: int64

In [9]:
final_df.groupby("city")["total_amount"].sum()

city
Bangalore    2206946.58
Chennai      1990513.03
Hyderabad    1889366.58
Pune         1924797.93
Name: total_amount, dtype: float64

In [10]:
final_df.groupby("cuisine")["total_amount"].sum()

cuisine
Chinese    1930504.65
Indian     1971412.58
Italian    2024203.80
Mexican    2085503.09
Name: total_amount, dtype: float64

In [11]:
final_df.groupby("membership")["total_amount"].mean()

membership
Gold       797.145556
Regular    805.158434
Name: total_amount, dtype: float64

In [12]:
final_df["total_amount"].sum()

np.float64(8011624.12)

In [13]:
final_df["total_amount"].mean()

np.float64(801.162412)

In [14]:
final_df[final_df["membership"]=="Gold"].groupby("city")["total_amount"].sum().sort_values(ascending=False)

city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [15]:
final_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [17]:
user_total = final_df.groupby("user_id")["total_amount"].sum()
count_users = (user_total > 1000).sum()
count_users

np.int64(2544)

In [53]:
final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0],
    labels=["3.0-3.5", "3.6-4.0", "4.1-4.5", "4.6-5.0"],
    include_lowest=True
)

rating_revenue = final_df.groupby("rating_range",observed=True)["total_amount"].sum().sort_values(ascending=False)
print(rating_revenue)


rating_range
4.6-5.0    2197030.75
3.0-3.5    2136772.70
4.1-4.5    1960326.26
3.6-4.0    1717494.41
Name: total_amount, dtype: float64


In [19]:
final_df[final_df["membership"]=="Gold"].groupby("city")["total_amount"].mean().sort_values(ascending=False)

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [20]:
final_df.groupby("cuisine").agg(restaurants=("restaurant_id","nunique"),revenue=("total_amount","sum")).sort_values(["restaurants","revenue"],ascending=[True,False])

,restaurants,revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [25]:
gold_pct = (final_df[final_df["membership"]=="Gold"].shape[0] /
            final_df.shape[0]) * 100

round(gold_pct)

50

In [51]:
restaurant_stats = final_df.groupby("restaurant_name_y").agg( total_orders=("order_id", "count"), avg_order_value=("total_amount", "mean") )
filtered = restaurant_stats[restaurant_stats["total_orders"] < 20]
highest = filtered.sort_values("avg_order_value", ascending=False).head(1) 
print(highest)

                   total_orders  avg_order_value
restaurant_name_y                               
Restaurant_294               13      1040.222308


In [31]:
final_df.groupby(["membership","cuisine"])["total_amount"].sum().sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [52]:
import pandas as pd
df=pd.read_csv("final_food_delivery_dataset.csv")
df["order_date"]=pd.to_datetime(df["order_date"],dayfirst=True)
df["quarter"]=df["order_date"].dt.quarter
df["quarter"]=df["order_date"].dt.quarter
quarter_revenue=df.groupby("quarter")["total_amount"].sum()
print(quarter_revenue)
high_quarter=quarter_revenue.idxmax()
print(high_quarter)

quarter
1    2010626.64
2    1945348.72
3    2037385.10
4    2018263.66
Name: total_amount, dtype: float64
3


In [41]:
gold_orders=df[df["membership"]=="Gold"]["order_id"].count()
print(gold_orders)

4987


In [42]:
hyd_revenue=df[df["city"]=="Hyderabad"]["total_amount"].sum()
print(round(hyd_revenue))

1889367


In [43]:
distinct_user=df["user_id"].nunique()
print(distinct_user)

2883


In [44]:
avg_gold_order_value=df[df["membership"]=="Gold"]["total_amount"].mean()
print(round(avg_gold_order_value))

797


In [45]:
high_rate_order=df[df["rating"]>=4.5]["order_id"].count()
print(high_rate_order)

3374


In [47]:
gold_city_revenue=df[df["membership"]=="Gold"].groupby("city")["total_amount"].sum()
top_city=gold_city_revenue.idxmax()
orders_top_city=df[(df["membership"]=="Gold")&(df["city"]==top_city)]["order_id"].count()
print(top_city)
print(orders_top_city)

Chennai
1337
